In [13]:
import requests_cache
from datetime import timedelta
import json
from info import get_address_info
from ens import address_from_ens, ens_from_address, other_ens_owned_by, domain_events
from transactions import get_transactions
from tokens import get_token_transfers
from time_series import get_time_series
from graph import count_transactions_by_address
from typing import List, Tuple, Dict

session = requests_cache.CachedSession('../cache',
                                       expire_after=timedelta(days=3))

# INPUT = "0x71B5b8Ec4D724849E76954A1475930dcE9c4B994"
INPUT = "kartik.eth"
input = INPUT

if input.endswith(".eth"):
    input = address_from_ens(session, input)
ens = ens_from_address(session, input)
other_ens_owned = other_ens_owned_by(session, input)
addresses_with_domain_events = domain_events(session, input)
transactions = get_transactions(session, input)
token_transfers = get_token_transfers(session, input)

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
df = pd.DataFrame(transactions + token_transfers)
# df = df[df['from'] == input]
df

,from,from_ens,to,to_ens,contract,chain,timestamp,symbol
0,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,kartik.eth,0x253553366Da8546fC250F225fe3d25d0C782303b,controller.ens.eth,True,eth,2025-03-05T20:45:59.000000Z,NaN
1,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,kartik.eth,0x283Af0B28c62C092C9727F1Ee09c02CA627EB7F5,None,True,eth,2025-03-05T20:41:47.000000Z,NaN
2,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,kartik.eth,0x283Af0B28c62C092C9727F1Ee09c02CA627EB7F5,None,True,eth,2025-03-05T20:40:35.000000Z,NaN
3,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,kartik.eth,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,None,True,eth,2025-02-08T03:13:35.000000Z,NaN
4,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,kartik.eth,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,None,True,eth,2025-02-08T03:11:47.000000Z,NaN
...,...,...,...,...,...,...,...,...
307,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,kartik.eth,0x8f61359578fe138F8444e08D225992bF2dA41c02,None,NaN,base,2024-05-11T23:58:29.000000Z,USDC
308,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,kartik.eth,0x8f61359578fe138F8444e08D225992bF2dA41c02,None,NaN,base,2024-05-11T23:53:47.000000Z,USDC
309,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,None,0x6b17ee547F38a210A43Dab7a72f65C9B6f69dcbD,None,NaN,polygon,2025-02-08T04:10:19.000000Z,USDC
310,0x53C61cfb8128ad59244E8c1D26109252ACe23d14,None,0xC990c9FCddF261464f5B5c096Dbd7707c2BA1540,None,NaN,polygon,2025-02-08T04:10:19.000000Z,USDC


In [27]:
df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
hour_counts = df['hour'].value_counts().sort_index()

# Ensure all 24 hours are represented (fill 0 where no data)
full_hour_counts = pd.Series([hour_counts.get(i, 0) for i in range(24)], index=range(24))
# Convert to numpy for rolling sum
counts_array = np.array(full_hour_counts.tolist())
# Duplicate the array to handle wrap-around for circular window
counts_double = np.concatenate([counts_array, counts_array[:7]])  # 24 + 7 = 31 elements
# Find 8-hour rolling sum and get the start index of the minimum sum
window_sums = np.convolve(counts_double, np.ones(8, dtype=int), 'valid')[:24]
min_start_hour = np.argmin(window_sums)
min_sum = window_sums[min_start_hour]

# Output result
end_hour = (min_start_hour + 8) % 24
print(f"Least interactions are between {min_start_hour:02d}:00 and {end_hour:02d}:00 with {min_sum} interactions.")
print(f"guessing timezone UTC-{min_start_hour}")
full_hour_counts.to_list()

Least interactions are between 10:00 and 18:00 with 40 interactions.
guessing timezone UTC-10


[24,
 3,
 16,
 15,
 14,
 24,
 20,
 13,
 6,
 11,
 3,
 10,
 2,
 1,
 3,
 12,
 7,
 2,
 17,
 16,
 37,
 24,
 10,
 22]

In [24]:
px.histogram(df, x='hour', nbins=24)